In [1]:
import numpy as np
import pandas as pd

Read in your label data:

In [2]:
labels = pd.read_csv('labels.csv')
labels.head(10)

,rad1,rad2,rad3,biopsy
0,benign,benign,benign,benign
1,benign,benign,benign,benign
2,benign,benign,benign,benign
3,benign,benign,benign,benign
4,benign,benign,cancer,benign
5,cancer,cancer,cancer,cancer
6,benign,benign,benign,benign
7,benign,benign,benign,benign
8,cancer,cancer,benign,cancer
9,benign,benign,cancer,benign


In [11]:
# Just checking there are no empty values or something
labels["rad1"].unique(), labels["rad2"].unique(), labels["rad3"].unique(), labels["biopsy"].unique()

(array(['benign', 'cancer'], dtype=object),
 array(['benign', 'cancer'], dtype=object),
 array(['benign', 'cancer'], dtype=object),
 array(['benign', 'cancer'], dtype=object))

In [23]:
for col in labels.columns:
    labels[col] = labels[col].apply(lambda x: 0 if x=="benign" else 1)

In [26]:
labels.head(10)

,rad1,rad2,rad3,biopsy
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,1,0
5,1,1,1,1
6,0,0,0,0
7,0,0,0,0
8,1,1,0,1
9,0,0,1,0


## Create your first ground truth as derived from biopsy labels: 

In [27]:
ground_truth_1 = labels["biopsy"]
ground_truth_1[:10]

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    1
9    0
Name: biopsy, dtype: int64

## Create your second truth by voting system from the three radiologists:

In [29]:
labels[['rad1', 'rad2', 'rad3']].head(10)

,rad1,rad2,rad3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,1
5,1,1,1
6,0,0,0
7,0,0,0
8,1,1,0
9,0,0,1


In [38]:
average = lambda x: sum(x)//2
ground_truth_2 = labels[['rad1', 'rad2', 'rad3']].apply(average, axis=1)

In [39]:
ground_truth_2.head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    1
9    0
dtype: int64

## Create your third ground truth by weighting the three radiologists:

In [52]:
weights = np.array([5, 10, 15])
weights_half = int(np.sum(weights)//2) + 1

weights_half

16

In [53]:
w_average = lambda x: weights.dot(x).sum()//weights_half

In [55]:
labels[['rad1', 'rad2', 'rad3']].apply(w_average, axis=1)[:10]

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    0
9    0
dtype: int64

In [56]:
ground_truth_3 = labels[['rad1', 'rad2', 'rad3']].apply(w_average, axis=1)

In [57]:
ground_truth_3.head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    0
9    0
dtype: int64

## Compare the three ground truths:

Here, just explore the three sets of labels you created and see how often they agree

In [59]:
ground_truths = pd.DataFrame({"gt1": ground_truth_1, "gt2": ground_truth_2, "gt3": ground_truth_3})

In [60]:
ground_truths[:10]

,gt1,gt2,gt3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,1,1,1
6,0,0,0
7,0,0,0
8,1,1,0
9,0,0,0


How good is `ground_truth_2` compared ot the biopsy

In [66]:
sum(list(map(lambda x: 1 if x else 0, list(ground_truth_1 == ground_truth_2))))/len(ground_truth_1)

0.85

How good is `ground_truth_3` compared ot the biopsy

In [67]:
sum(list(map(lambda x: 1 if x else 0, list(ground_truth_1 == ground_truth_3))))/len(ground_truth_1)

0.8